In [ ]:
from torchvision import datasets

import numpy as np

# preprocessing
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split

# model
import torch
from torch import nn
from torch.optim import Adam

# 1. Load Data

In [ ]:
def load_data(data_dir):
    """ Create train and test pytorch dataset objects from CIFAR10.
    
    The following tranformations are applied on CIFAR10:
        * image pixels to [0, 1] range,
        * normalization (by substracting mean and dividing with std; [-1, 1] range)
    
    Args:
        data_dir:
            directory where data will be saved, as a string.
    
    Returns:
        train and test dataset, as pytorch dataset objects.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR10(root=data_dir,
                                train=True, 
                                download=True, 
                                transform=transform)

    testset = datasets.CIFAR10(root=data_dir, 
                               train=False, 
                               download=True, 
                               transform=transform)

    return trainset, testset

In [ ]:
training_data, test_data = load_data(data_dir='cifar10')

print(f'\nTraining data:\n--------------\n{training_data}')
print(f'Test data:\n--------------\n{test_data}')

# 2. LeNet-5

In [ ]:
class LeNet5(nn.Module):
    
    def __init__(self, output_size):
        super(LeNet5, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=output_size)
        
    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        
        # flatten all dimensions except batch
        x = torch.flatten(x, 1) 
        
        logits = torch.tanh(self.fc1(x))
        logits = torch.tanh(self.fc2(logits))
        logits = self.fc3(logits)
        
        return logits

# 3. Fit and Predict

In [ ]:
def fit(dataloader, model, loss_fn, optimizer, print_loss=False):
    """ Fit neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
        optimizer:
            optimizer function, as pytorch object.
        print_loss:
            print loss on every batch, as boolean (default False)
    """
    size = len(dataloader.dataset)
    model.train()  # put on train mode
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        
        # compute prediction
        pred = model(X)
        
        # compute loss
        loss = loss_fn(pred, Y)

        # reset the gradients
        optimizer.zero_grad()
        
        # backpropagate
        loss.backward()
        
        # update parameters
        optimizer.step()

        if print_loss and batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def predict(dataloader, model, loss_fn):
    """ Predict with neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
    """
    size = len(dataloader.dataset)
    
    test_loss = 0
    correct = 0
    
    model.eval()  # put on evaluation mode
    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)
            
            pred = model(X)
            
            test_loss += loss_fn(pred, Y).item()
            
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"average loss: {test_loss:>8f} \naccuracy: {(100*correct):>0.1f}%\n")

# 4. Final Model

In [ ]:
batch_size = 64

training_data, test_data = load_data(data_dir='cifar10')

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
num_labels = 10

model = LeNet5(output_size=num_labels)
model.to(device)

print(f'Model architecture:\n{model}')

In [ ]:
learning_rate = 0.001
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    fit(train_dataloader, model, loss_fn, optimizer, print_loss=False)
    
    print('\nTrain:\n-------')
    predict(train_dataloader, model, loss_fn)
    
    print('\nTest:\n-------')
    predict(test_dataloader, model, loss_fn)
    
    print(f"-------------------------------")